# SIRF reconstruction using data simulated with SIMIND ###
This notebook lays out a simple simulation and reconstruction using SIMIND and SIRF \
Please see Rebecca Gillen's instructions for a more in depth guide

### simind can be donwloaded from https://simind.blogg.lu.se/downloads/
There are reasonably straight forward instructions to be followed for Windows/Mac/Linux
### SIRF can be downloaded from https://github.com/SyneRBI/SIRF
There are more complicated, but logical instructions for building the required packaged and libraries at https://github.com/SyneRBI/SIRF/wiki/Installation-instructions

In [ ]:
# import sirf
import sirf.STIR as STIR

msg = STIR.MessageRedirector("info.txt", "warnings.txt", "error.txt") # redirects error, information and warning messages to file

Below we define two functions (the first of which should hopefully become necessary) in order to set up a voxel-based phantom (to use SIMIND's language)

In [ ]:
def crop_image(templ_sino, image, nx, ny, nz, slice = None):
    """Crop ImageData from (vol_z,vol_y,vol_x) to (nz,ny,nx)"""
    vol = image.as_array()
    vol_dims = vol.shape
    x_origin = vol_dims[2]//2
    y_origin = vol_dims[1]//2
    if slice is None:
        z_origin = vol_dims[0]//2
    else:
        z_origin = slice
    
    vol = vol[z_origin-nz//2:z_origin+nz//2+nz%2,y_origin-ny//2:y_origin+ny//
              2+ny%2,x_origin-nx//2:x_origin+nx//2+nx%2]
    im = STIR.ImageData(templ_sino)
    dim=(nz,ny,nx)
    vol = vol.reshape(dim)
    voxel_size=im.voxel_sizes()
    im.initialise(dim,voxel_size)
    im.fill(vol)
    return im

In [ ]:
def create_sample_image(image):
    '''fill the image with some geometric shapes to create a simple phantom image'''
    im_shape = image.shape
    image.fill(0)
    
    # create a body-like ellipsoid shape
    shape = STIR.EllipticCylinder()
    shape.set_length(400)
    shape.set_radii((im_shape[1]//3*4, im_shape[2]//2*3))
    shape.set_origin((0, 0, 0))

    # add the shape to the image
    image.add_shape(shape, scale = 150)

    # add some lung-like shapes shape
    shape.set_radii((im_shape[1], im_shape[2]//2))
    shape.set_origin((0,0, -im_shape[2]//3*2))
    image.add_shape(shape, scale = -75)

    shape.set_origin((0, 0, im_shape[2]//3*2))
    image.add_shape(shape, scale = -75)

    # a spine-like shape
    shape.set_radii((im_shape[1]//4, im_shape[2]//4))
    shape.set_origin((0, im_shape[1]//5*4, 0))
    image.add_shape(shape, scale = 75)

    # and a lung tumour-like shape
    shape.set_radii((im_shape[1]//5, im_shape[2]//5))
    shape.set_origin((0, -im_shape[1]//5, im_shape[2]//2))
    image.add_shape(shape, scale = 100)
    
    return image

We are now ready to create our image. We require only a template sinogram containing information such as the image dimensions and number of projections (among many other things - take a look at the .hs file)

In [ ]:
templ_sino = STIR.AcquisitionData("./template_sinogram.hs") # template sirf acquisition data
im = create_sample_image(templ_sino.create_uniform_image(0)) # use the template to create an empty image

### I'm currently unsure why this is necessary. It requires further investigastion ###
im = crop_image(templ_sino, im, templ_sino.dimensions()[3], templ_sino.dimensions()[2], templ_sino.dimensions()[1])

im = im.zoom_image((0.5,1,1)) # zoom the image along the z axis. This line is required because SIRF was originally set up for PET data with a 180 degree acquisition

att_STIR = im/1000 # approximate attenuation coefficient image used in STIR (/cm)
att_SIMIND = att_STIR/0.15*1000 # approximate density image used in SIMIND (mg/cm^3)
att_none = im.get_uniform_copy(0) # zero density map to investigate the effects on SIMIND reconstruction

And we have a simple image phantom

In [ ]:
im.show()

We now write these images to file.
SIMIND requires a .dmi file for an attenuation (density) image and a .smi for an emission (source) image\
We can easily do this using the ImageData.write() method

In [ ]:
im.write("./image.smi")
att_STIR.write("./attenuation_stir.hv")
att_SIMIND.write("./attenuation_simind.dmi")
att_none.write("./attenuation_none.dmi")

We're now ready to simulate our emission data. We have a .smc file containing information about the simulation. Please read the simind manual to learn about the many different options available. These options can be altered using either the change command (type "change input.smc into the terminal) or using switches \
The syntax for reconstruction is as follows:
`simind input_file_prefix outpute_files_prefix`
This can be followed by switches seperated by forward slashes such as below \*\
\* Unfortunately this causes some trible with Linux & MacOS file directories. The SIMIND manual claims that two backslashed '\\' can be used in place of a forward slash that is part of a file directory, but I haven't found this to be the case

The following bash command defines a .smc file `input.smc` follwed by a prefix for output files `output` \
Switches are then used to define:
* /NN: a multiplier for the number of histories per projection (which is calculated using the sum of all voxel values)
* /PX: defines the image pixel size
* /FS: defines the prefix for the .smi emission image file
* /FD: defines the prefix for rhe .dmi attenuation image file

In [ ]:
%%bash
simind input output/NN:0.001/PX:0.4/FS:image/FD:attenuation_simind

And (assuming the preious cell ran) we have now simulated our SPECT data!\
Next we need to get this data into a format the SIRF will recognise. Luckily we have a script ready that does this for us.
This script changes a few lines in the data's header file and the header file suffix. Differences between the conventions of interfiles in SIMIND and STIR/SIRF can be found in Rebecca's notes.

In [ ]:
%%bash
sh ./convertSIMINDToSTIR.sh output.h00

We can now view the resulting sinogram

In [ ]:
simulated_data = STIR.AcquisitionData("./output.hs")
simulated_data.show()

OK, so now we have our projection data in a format that SIRF likes, we can go about reconstructing the data. In this notebook we do this as simply as possible with a back prokection. \
In order to do this we first need to create our acqusition model matrix

In [ ]:
acq_model_matrix = STIR.SPECTUBMatrix()
acq_model_matrix.set_attenuation_image(att_STIR) # set the attenuation image for reconstruction 
acq_model_matrix.set_resolution_model(0.1,0.1,full_3D=False) # where we have defnied our collimator blurring as a gaussian with SD 0.1mm and a collimator slope of 0.1mm

We then use this matrix to set up an acqusition model. This requires the set-Up() method to be run and needs a projection and image template 

In [ ]:
am = STIR.AcquisitionModelUsingMatrix(acq_model_matrix)
am.set_up(simulated_data, im)

And we can finally backproject the data

In [ ]:
res = am.backward(simulated_data)
res.show()

Below is a forward and backproject of the data using SIRF only for comparison (obvisouly this doesn't model noise in any way)

In [ ]:
forward_projected_data = am.forward(im)
forward_projected_data.show()

In [ ]:
back_projected_image = am.backward(forward_projected_data)
back_projected_image.show()

This last cell removes any temporary image files that SIRF occassionally misses

In [ ]:
import glob
import os

for f in glob.glob(("./tmp*")):
    os.remove(f)